In [263]:
import pandas as pd
import os

In [264]:
master = pd.read_pickle(os.path.join(".","pickle_files", "master.pickle"))
team_splits = pd.read_pickle(os.path.join(".", "pickle_files", "team_splits.pickle"))

In [265]:
master.head()

,firstName,lastName,pos,birthYear,birthMon,birthDay,birthCountry,birthState,birthCity
playerID,,,,,,,,,
aaltoan01,Antti,Aalto,C,1975.0,3.0,4.0,Finland,NaN,Lappeenranta
abdelju01,Justin,Abdelkader,L,1987.0,2.0,25.0,USA,MI,Muskegon
abidra01,Ramzi,Abid,L,1980.0,3.0,24.0,Canada,QC,Montreal
abrahth01,Thommy,Abrahamsson,D,1947.0,4.0,12.0,Sweden,NaN,Leksand
actonke01,Keith,Acton,C,1958.0,4.0,15.0,Canada,ON,Stouffville


In [266]:
master = master.assign(birthDate= pd.to_datetime({"year": master.birthYear, "month":master.birthMon, "day":master.birthDay}))

In [267]:
master.drop(columns=["birthYear", "birthMon", "birthDay"], inplace=True)

In [268]:
master.head()

,firstName,lastName,pos,birthCountry,birthState,birthCity,birthDate
playerID,,,,,,,
aaltoan01,Antti,Aalto,C,Finland,NaN,Lappeenranta,1975-03-04
abdelju01,Justin,Abdelkader,L,USA,MI,Muskegon,1987-02-25
abidra01,Ramzi,Abid,L,Canada,QC,Montreal,1980-03-24
abrahth01,Thommy,Abrahamsson,D,Sweden,NaN,Leksand,1947-04-12
actonke01,Keith,Acton,C,Canada,ON,Stouffville,1958-04-15


In [269]:
ts = master.birthDate[0]
ts

Timestamp('1975-03-04 00:00:00')

In [270]:
print(u"{} , {} {} {}".format(ts.day_name(), ts.day, ts.month_name(), ts.year) )

Tuesday , 4 March 1975


In [271]:
tsz = ts.tz_localize("Europe/London")
tsz

Timestamp('1975-03-04 00:00:00+0000', tz='Europe/London')

In [272]:
tsz.tz_convert("America/Toronto")

Timestamp('1975-03-03 19:00:00-0500', tz='America/Toronto')

In [273]:
years = master.birthDate.dt.year
years.head()

playerID
aaltoan01    1975
abdelju01    1987
abidra01     1980
abrahth01    1947
actonke01    1958
Name: birthDate, dtype: int64

In [274]:
strings = master.birthDate.dt.strftime("%Y-%m-%d-%a")
strings.head()

playerID
aaltoan01    1975-03-04-Tue
abdelju01    1987-02-25-Wed
abidra01     1980-03-24-Mon
abrahth01    1947-04-12-Sat
actonke01    1958-04-15-Tue
Name: birthDate, dtype: object

In [275]:
strings = master.birthDate.dt.strftime("%Y-%m-%d")
strings.head()

playerID
aaltoan01    1975-03-04
abdelju01    1987-02-25
abidra01     1980-03-24
abrahth01    1947-04-12
actonke01    1958-04-15
Name: birthDate, dtype: object

In [276]:
dates = pd.to_datetime(strings)
dates.head()

playerID
aaltoan01   1975-03-04
abdelju01   1987-02-25
abidra01    1980-03-24
abrahth01   1947-04-12
actonke01   1958-04-15
Name: birthDate, dtype: datetime64[ns]

In [277]:
strings = strings.str.replace("-", "xx")
strings.head()

playerID
aaltoan01    1975xx03xx04
abdelju01    1987xx02xx25
abidra01     1980xx03xx24
abrahth01    1947xx04xx12
actonke01    1958xx04xx15
Name: birthDate, dtype: object

In [278]:
try: 
    pd.to_datetime(strings)
except Exception as e:
    print(e)

('Unknown string format:', '1975xx03xx04')


In [279]:
parsed = pd.to_datetime(strings, format="%Yxx%mxx%d")
parsed.head()

playerID
aaltoan01   1975-03-04
abdelju01   1987-02-25
abidra01    1980-03-24
abrahth01   1947-04-12
actonke01   1958-04-15
Name: birthDate, dtype: datetime64[ns]

In [280]:
string_error = strings.copy()
string_error.iloc[3] = "xx23"
try: 
    pd.to_datetime(string_error, format="%Yxx%mxx%d")
except Exception as e:
    print(e)

time data 'xx23' does not match format '%Yxx%mxx%d' (match)


In [281]:
parsed = pd.to_datetime(string_error, format="%Yxx%mxx%d", errors="coerce")
parsed.head()
# NaT stands for Not A Time

playerID
aaltoan01   1975-03-04
abdelju01   1987-02-25
abidra01    1980-03-24
abrahth01          NaT
actonke01   1958-04-15
Name: birthDate, dtype: datetime64[ns]

Don't use errors="ignore" it ignores the erros and gives unpredictable results

In [282]:
parsed = pd.to_datetime(string_error, format="%Yxx%mxx%d", errors="ignore")
parsed.head()
# NaT stands for Not A Time

playerID
aaltoan01    1975xx03xx04
abdelju01    1987xx02xx25
abidra01     1980xx03xx24
abrahth01            xx23
actonke01    1958xx04xx15
Name: birthDate, dtype: object

In [283]:
birth_dates = pd.Series(master.index, index=master.birthDate)
birth_dates.head()

birthDate
1975-03-04    aaltoan01
1987-02-25    abdelju01
1980-03-24     abidra01
1947-04-12    abrahth01
1958-04-15    actonke01
Name: playerID, dtype: object

In [284]:
birth_dates.index

DatetimeIndex(['1975-03-04', '1987-02-25', '1980-03-24', '1947-04-12',
               '1958-04-15', '1990-06-18', '1961-05-05', '1977-03-20',
               '1977-04-26', '1960-05-31',
               ...
               '1987-09-01', '1963-05-08', '1987-03-03', '1987-02-14',
               '1970-07-22', '1978-06-16', '1987-09-01', '1992-01-16',
               '1954-04-16', '1978-01-21'],
              dtype='datetime64[ns]', name='birthDate', length=4627, freq=None)

**Always sort the index for time series analysis**

In [285]:
birth_dates = birth_dates.sort_index()
birth_dates.head()

birthDate
1940-01-27    harpete01
1940-03-22     keonda01
1940-10-03    ratelje01
1942-02-20    esposph01
1943-04-23    esposto01
Name: playerID, dtype: object

**Very tricky one Keep this in mind**

In [286]:
print(birth_dates[1980])  # this will give the index position
print(birth_dates.iloc[1980])  # this will give the index position
born_1980 = birth_dates["1980"]  # this will get all players born in 1980
born_1980.head()

klemmjo01
klemmjo01


birthDate
1980-01-02    zalesmi01
1980-01-10    stanara01
1980-01-13    bouckty01
1980-01-13     ruppmi01
1980-01-15    papinju01
Name: playerID, dtype: object

In [287]:
birth_dates["1980-07"].head()

birthDate
1980-07-03    miettan01
1980-07-05    huntetr01
1980-07-08    chouier01
1980-07-15    cheecjo01
1980-07-17    millery01
Name: playerID, dtype: object

In [288]:
birth_dates["1980-07-24"].head()

birthDate
1980-07-24    jillsje01
1980-07-24    printda01
Name: playerID, dtype: object

In [289]:
birth_dates["1989-07-20" : "1989-08-15"].head()  # both upper and lower limits are inclusive

birthDate
1989-07-24    almonco01
1989-07-25    troppco01
1989-08-10    gagnesa01
1989-08-14    turriky01
1989-08-15    voracja01
Name: playerID, dtype: object

# Timedelta

In [290]:
diff = birth_dates.index[1] - birth_dates.index[0]
diff

Timedelta('55 days 00:00:00')

In [291]:
birth_dates.index[2]

Timestamp('1940-10-03 00:00:00')

In [292]:
birth_dates.index[2] + diff

Timestamp('1940-11-27 00:00:00')

In [293]:
birth_dates.index - diff

DatetimeIndex(['1939-12-03', '1940-01-27', '1940-08-09', '1941-12-27',
               '1943-02-27', '1944-01-19', '1944-11-14', '1944-12-05',
               '1945-02-01', '1945-04-30',
               ...
               '1992-06-06', '1992-07-15', '1992-08-11', '1992-09-02',
               '1992-09-18', '1992-09-29', '1992-10-13', '1993-01-19',
               '1993-02-16', '1993-02-22'],
              dtype='datetime64[ns]', name='birthDate', length=4627, freq=None)

In [294]:
diff.components

Components(days=55, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0, nanoseconds=0)

In [295]:
diff = diff + "5H" - "10M"
diff

Timedelta('55 days 04:50:00')

In [296]:
diff.round("10D")

Timedelta('60 days 00:00:00')

In [297]:
diff.round("3H")

Timedelta('55 days 06:00:00')

In [298]:
diff.round("1H")

Timedelta('55 days 05:00:00')

In [299]:
birth_dates.index
# check the dytpe of the output

DatetimeIndex(['1940-01-27', '1940-03-22', '1940-10-03', '1942-02-20',
               '1943-04-23', '1944-03-14', '1945-01-08', '1945-01-29',
               '1945-03-28', '1945-06-24',
               ...
               '1992-07-31', '1992-09-08', '1992-10-05', '1992-10-27',
               '1992-11-12', '1992-11-23', '1992-12-07', '1993-03-15',
               '1993-04-12', '1993-04-18'],
              dtype='datetime64[ns]', name='birthDate', length=4627, freq=None)

In [300]:
birth_dates=birth_dates.to_period(freq="D")
birth_dates.index
# check the dytpe of the output

PeriodIndex(['1940-01-27', '1940-03-22', '1940-10-03', '1942-02-20',
             '1943-04-23', '1944-03-14', '1945-01-08', '1945-01-29',
             '1945-03-28', '1945-06-24',
             ...
             '1992-07-31', '1992-09-08', '1992-10-05', '1992-10-27',
             '1992-11-12', '1992-11-23', '1992-12-07', '1993-03-15',
             '1993-04-12', '1993-04-18'],
            dtype='period[D]', name='birthDate', length=4627, freq='D')

In [301]:
birth_dates.index[0]

Period('1940-01-27', 'D')

In [302]:
start = birth_dates.index[0].start_time
print(start)
end = birth_dates.index[0].end_time
print(end)

1940-01-27 00:00:00
1940-01-27 23:59:59.999999999


In [303]:
start < (start + pd.Timedelta("5H")) < end

True

In [304]:
start < (start + pd.Timedelta("49H")) < end

False

In [305]:
birth_dates.index.year

Int64Index([1940, 1940, 1940, 1942, 1943, 1944, 1945, 1945, 1945, 1945,
            ...
            1992, 1992, 1992, 1992, 1992, 1992, 1992, 1993, 1993, 1993],
           dtype='int64', name='birthDate', length=4627)

**Remember birth_dates[1940] is location not the year. birth_dates["1940"] is year**

In [306]:
birth_dates["1940"]

birthDate
1940-01-27    harpete01
1940-03-22     keonda01
1940-10-03    ratelje01
1942-02-20    esposph01
Freq: D, Name: playerID, dtype: object

In [307]:
birth_dates["1940-03-22"]

'keonda01'

# Advanced Strategies

resample is kind of group by for time series data

In [308]:
birth_dates.resample("1M")

PeriodIndexResampler [freq=<MonthEnd>, axis=0, closed=right, label=right, convention=start, base=0]

In [309]:
birth_dates.resample("1M").count().head()

birthDate
1940-01    1
1940-02    0
1940-03    1
1940-04    0
1940-05    0
Freq: M, Name: playerID, dtype: int64

In [310]:
birth_dates.resample("2M").count().head()

birthDate
1940-01    1
1940-03    1
1940-05    0
1940-07    0
1940-09    1
Freq: 2M, Name: playerID, dtype: int64

In [311]:
birth_dates.resample("1Q").count().head()

birthDate
1940Q1    2
1940Q2    0
1940Q3    0
1940Q4    1
1941Q1    0
Freq: Q-DEC, Name: playerID, dtype: int64

In [320]:
birth_dates.resample("1W").count().head()

birthDate
1940-01-22/1940-01-28    1
1940-01-29/1940-02-04    0
1940-02-05/1940-02-11    0
1940-02-12/1940-02-18    0
1940-02-19/1940-02-25    0
Freq: W-SUN, Name: playerID, dtype: int64

In [321]:
from pandas.tseries.offsets import BDay
p = birth_dates.index[2]
print(p.to_timestamp())                  # We are converting this to time stamp because Bday doesn't work well with Period Index
print(p.to_timestamp() + BDay(9)) 
# this will add 9 business days 


1940-10-03 00:00:00
1940-10-16 00:00:00


In [322]:
birth_dates.head()

birthDate
1940-01-27    harpete01
1940-03-22     keonda01
1940-10-03    ratelje01
1942-02-20    esposph01
1943-04-23    esposto01
Freq: D, Name: playerID, dtype: object

In [323]:
# shift freq should always match with index freq
birth_dates.shift(1, freq="D").head()

birthDate
1940-01-28    harpete01
1940-03-23     keonda01
1940-10-04    ratelje01
1942-02-21    esposph01
1943-04-24    esposto01
Freq: D, Name: playerID, dtype: object

In [324]:
try:
    birth_dates.shift(1, freq="M").head()
except Exception as e:
    print(e)
# you have to modify the index to period index to Month

Given freq M does not match PeriodIndex freq D


In [325]:
# this will move the row
birth_dates.shift(1).head()

birthDate
1940-01-27          NaN
1940-03-22    harpete01
1940-10-03     keonda01
1942-02-20    ratelje01
1943-04-23    esposph01
Freq: D, Name: playerID, dtype: object

In [326]:
team_splits.head()

,year,tmID,SepW,SepL,SepT,SepOL,OctW,OctL,OctT,OctOL,...,FebOL,MarW,MarL,MarT,MarOL,AprW,AprL,AprT,AprOL,name
0,1980,BOS,NaN,NaN,NaN,NaN,2.0,7.0,1.0,NaN,...,NaN,8.0,4.0,3.0,NaN,1.0,2.0,0.0,NaN,Boston Bruins
1,1980,BUF,NaN,NaN,NaN,NaN,5.0,2.0,2.0,NaN,...,NaN,8.0,6.0,1.0,NaN,1.0,1.0,1.0,NaN,Buffalo Sabres
2,1980,CAL,NaN,NaN,NaN,NaN,5.0,4.0,2.0,NaN,...,NaN,6.0,6.0,2.0,NaN,2.0,0.0,0.0,NaN,Calgary Flames
3,1980,CHI,NaN,NaN,NaN,NaN,6.0,4.0,2.0,NaN,...,NaN,4.0,5.0,4.0,NaN,1.0,1.0,1.0,NaN,Chicago Black Hawks
4,1980,COR,NaN,NaN,NaN,NaN,4.0,4.0,2.0,NaN,...,NaN,4.0,9.0,1.0,NaN,0.0,1.0,2.0,NaN,Colorado Rockies


In [ ]:
team_splits.drop(columns="tmID", inplace=True)

In [336]:

team_splits=team_splits.set_index(["name", "year"])
team_splits.head()

,,SepW,SepL,SepT,SepOL,OctW,OctL,OctT,OctOL,NovW,NovL,...,FebT,FebOL,MarW,MarL,MarT,MarOL,AprW,AprL,AprT,AprOL
name,year,,,,,,,,,,,,,,,,,,,,,
Boston Bruins,1980,NaN,NaN,NaN,NaN,2.0,7.0,1.0,NaN,5.0,3.0,...,2.0,NaN,8.0,4.0,3.0,NaN,1.0,2.0,0.0,NaN
Buffalo Sabres,1980,NaN,NaN,NaN,NaN,5.0,2.0,2.0,NaN,8.0,4.0,...,3.0,NaN,8.0,6.0,1.0,NaN,1.0,1.0,1.0,NaN
Calgary Flames,1980,NaN,NaN,NaN,NaN,5.0,4.0,2.0,NaN,5.0,5.0,...,1.0,NaN,6.0,6.0,2.0,NaN,2.0,0.0,0.0,NaN
Chicago Black Hawks,1980,NaN,NaN,NaN,NaN,6.0,4.0,2.0,NaN,3.0,8.0,...,4.0,NaN,4.0,5.0,4.0,NaN,1.0,1.0,1.0,NaN
Colorado Rockies,1980,NaN,NaN,NaN,NaN,4.0,4.0,2.0,NaN,5.0,6.0,...,2.0,NaN,4.0,9.0,1.0,NaN,0.0,1.0,2.0,NaN


In [337]:
month = team_splits.columns.map(lambda x: x[:3])
metric = team_splits.columns.map(lambda x: x[3:])
print(month)
print(metric)

Index(['Sep', 'Sep', 'Sep', 'Sep', 'Oct', 'Oct', 'Oct', 'Oct', 'Nov', 'Nov',
       'Nov', 'Nov', 'Dec', 'Dec', 'Dec', 'Dec', 'Jan', 'Jan', 'Jan', 'Jan',
       'Feb', 'Feb', 'Feb', 'Feb', 'Mar', 'Mar', 'Mar', 'Mar', 'Apr', 'Apr',
       'Apr', 'Apr'],
      dtype='object')
Index(['W', 'L', 'T', 'OL', 'W', 'L', 'T', 'OL', 'W', 'L', 'T', 'OL', 'W', 'L',
       'T', 'OL', 'W', 'L', 'T', 'OL', 'W', 'L', 'T', 'OL', 'W', 'L', 'T',
       'OL', 'W', 'L', 'T', 'OL'],
      dtype='object')


In [338]:
midx = pd.MultiIndex.from_arrays([month, metric])
midx

MultiIndex(levels=[['Apr', 'Dec', 'Feb', 'Jan', 'Mar', 'Nov', 'Oct', 'Sep'], ['L', 'OL', 'T', 'W']],
           codes=[[7, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 4, 4, 4, 4, 0, 0, 0, 0], [3, 0, 2, 1, 3, 0, 2, 1, 3, 0, 2, 1, 3, 0, 2, 1, 3, 0, 2, 1, 3, 0, 2, 1, 3, 0, 2, 1, 3, 0, 2, 1]])

In [339]:
team_splits.columns = midx
team_splits.head()

Sep              Oct                Nov       ...  \
                           W   L   T  OL    W    L    T  OL    W    L  ...   
name                year                                               ...   
Boston Bruins       1980 NaN NaN NaN NaN  2.0  7.0  1.0 NaN  5.0  3.0  ...   
Buffalo Sabres      1980 NaN NaN NaN NaN  5.0  2.0  2.0 NaN  8.0  4.0  ...   
Calgary Flames      1980 NaN NaN NaN NaN  5.0  4.0  2.0 NaN  5.0  5.0  ...   
Chicago Black Hawks 1980 NaN NaN NaN NaN  6.0  4.0  2.0 NaN  3.0  8.0  ...   
Colorado Rockies    1980 NaN NaN NaN NaN  4.0  4.0  2.0 NaN  5.0  6.0  ...   

                          Feb      Mar                Apr                
                            T  OL    W    L    T  OL    W    L    T  OL  
name                year                                                 
Boston Bruins       1980  2.0 NaN  8.0  4.0  3.0 NaN  1.0  2.0  0.0 NaN  
Buffalo Sabres      1980  3.0 NaN  8.0  6.0  1.0 NaN  1.0  1.0  1.0 NaN  
Calgary Flames      1980  1.0 NaN  6.0  6.0  2.0 NaN  2.0  0.0  0.0 NaN  
Chicago Black Hawks 1980  4.0 NaN  4.0  5.0  4.0 NaN  1.0  1.0  1.0 NaN  
Colorado Rockies    1980  2.0 NaN  4.0  9.0  1.0 NaN  0.0  1.0  2.0 NaN  

[5 rows x 32 columns]

In [348]:
team_splits.columns.levels[0].name= "month"
team_splits.columns.levels[1].name= "metric"


In [349]:
team_splits.head()

month                    Sep              Oct                Nov       ...  \
metric                     W   L   T  OL    W    L    T  OL    W    L  ...   
name                year                                               ...   
Boston Bruins       1980 NaN NaN NaN NaN  2.0  7.0  1.0 NaN  5.0  3.0  ...   
Buffalo Sabres      1980 NaN NaN NaN NaN  5.0  2.0  2.0 NaN  8.0  4.0  ...   
Calgary Flames      1980 NaN NaN NaN NaN  5.0  4.0  2.0 NaN  5.0  5.0  ...   
Chicago Black Hawks 1980 NaN NaN NaN NaN  6.0  4.0  2.0 NaN  3.0  8.0  ...   
Colorado Rockies    1980 NaN NaN NaN NaN  4.0  4.0  2.0 NaN  5.0  6.0  ...   

month                     Feb      Mar                Apr                
metric                      T  OL    W    L    T  OL    W    L    T  OL  
name                year                                                 
Boston Bruins       1980  2.0 NaN  8.0  4.0  3.0 NaN  1.0  2.0  0.0 NaN  
Buffalo Sabres      1980  3.0 NaN  8.0  6.0  1.0 NaN  1.0  1.0  1.0 NaN  
Calgary Flames      1980  1.0 NaN  6.0  6.0  2.0 NaN  2.0  0.0  0.0 NaN  
Chicago Black Hawks 1980  4.0 NaN  4.0  5.0  4.0 NaN  1.0  1.0  1.0 NaN  
Colorado Rockies    1980  2.0 NaN  4.0  9.0  1.0 NaN  0.0  1.0  2.0 NaN  

[5 rows x 32 columns]

In [355]:
team_splits = team_splits.stack(level="month")
team_splits.head()

metric                      L  OL    T    W
name          year month                   
Boston Bruins 1980 Apr    2.0 NaN  0.0  1.0
                   Dec    6.0 NaN  1.0  6.0
                   Feb    4.0 NaN  2.0  6.0
                   Jan    4.0 NaN  1.0  9.0
                   Mar    4.0 NaN  3.0  8.0

In [357]:
team_splits=team_splits.reset_index()
team_splits.head()

metric,name,year,month,L,OL,T,W
0,Boston Bruins,1980,Apr,2.0,NaN,0.0,1.0
1,Boston Bruins,1980,Dec,6.0,NaN,1.0,6.0
2,Boston Bruins,1980,Feb,4.0,NaN,2.0,6.0
3,Boston Bruins,1980,Jan,4.0,NaN,1.0,9.0
4,Boston Bruins,1980,Mar,4.0,NaN,3.0,8.0


In [359]:
starting_date = team_splits.apply(lambda x: pd.Period("{}-{}".format(x.year, x.month)), axis=1) # axix 1 for row
starting_date.head()

0    1980-04
1    1980-12
2    1980-02
3    1980-01
4    1980-03
dtype: period[M]

In [360]:
starting_date[1]

Period('1980-12', 'M')

In [364]:
team_splits.drop(columns="year", inplace=True)
team_splits = team_splits.assign(month = starting_date)
team_splits.head()

metric,name,month,L,OL,T,W
0,Boston Bruins,1980-04,2.0,NaN,0.0,1.0
1,Boston Bruins,1980-12,6.0,NaN,1.0,6.0
2,Boston Bruins,1980-02,4.0,NaN,2.0,6.0
3,Boston Bruins,1980-01,4.0,NaN,1.0,9.0
4,Boston Bruins,1980-03,4.0,NaN,3.0,8.0


In [365]:
team_splits = team_splits.set_index(["month"])
team_splits.head()

metric,name,L,OL,T,W
month,,,,,
1980-04,Boston Bruins,2.0,NaN,0.0,1.0
1980-12,Boston Bruins,6.0,NaN,1.0,6.0
1980-02,Boston Bruins,4.0,NaN,2.0,6.0
1980-01,Boston Bruins,4.0,NaN,1.0,9.0
1980-03,Boston Bruins,4.0,NaN,3.0,8.0


In [368]:
team_splits.columns.name= None

In [369]:
team_splits.head()

,name,L,OL,T,W
month,,,,,
1980-04,Boston Bruins,2.0,NaN,0.0,1.0
1980-12,Boston Bruins,6.0,NaN,1.0,6.0
1980-02,Boston Bruins,4.0,NaN,2.0,6.0
1980-01,Boston Bruins,4.0,NaN,1.0,9.0
1980-03,Boston Bruins,4.0,NaN,3.0,8.0


In [372]:
team_splits.groupby(["name"]).resample("Q").mean().head()

L   OL   T         W
name          month                              
Anaheim Ducks 2006Q1  4.333333  2.0 NaN  6.333333
              2006Q2  0.000000  2.0 NaN  1.000000
              2006Q3       NaN  NaN NaN       NaN
              2006Q4  2.333333  2.0 NaN  9.333333
              2007Q1  3.333333  1.0 NaN  8.666667

In [373]:
quarter_means = team_splits.groupby(["name"]).resample("Q").mean().head()
quarter_means.head()

L   OL   T         W
name          month                              
Anaheim Ducks 2006Q1  4.333333  2.0 NaN  6.333333
              2006Q2  0.000000  2.0 NaN  1.000000
              2006Q3       NaN  NaN NaN       NaN
              2006Q4  2.333333  2.0 NaN  9.333333
              2007Q1  3.333333  1.0 NaN  8.666667

In [ ]:
team_splits.to_pickle(os.path.join(".", "pickle_files", "team_splits_period.pickle"))
birth_dates.to_pickle(os.path.join(".", "pickle_files", ""))